<a href="https://colab.research.google.com/github/jovitaand/Sentimental-Analysis-on-Movie-Reviews-/blob/main/Sentiment_Analysis_on_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Definition:
The goal of this project is to develop a model capable of categorizing the sentiment expressed in movie reviews into five categories: very negative, negative, neutral, positive, and very positive. This task is significant as it helps in understanding customer perceptions and can be utilized for services such as recommendation systems or market analysis.

In [1]:
#import libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
sns.set(style='whitegrid')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix
from collections import defaultdict
from collections import Counter
import re
import gensim
import string
import tensorflow as tf
import warnings
warnings.simplefilter('ignore')
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ModuleNotFoundError: No module named 'wordcloud'

#Data Preparation

In [ ]:
import zipfile
import os

# Path to the uploaded zip file
zip_path = '/content/kagglemoviereviews.zip'
extract_folder = '/content/kaggle_movie_reviews'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# List the files extracted
extracted_files = os.listdir(extract_folder)
extracted_files


In [ ]:
# List the files in the 'kagglemoviereviews' directory
kaggle_reviews_path = os.path.join(extract_folder, 'kagglemoviereviews')
review_files = os.listdir(kaggle_reviews_path)
review_files


In [ ]:
# List the files in the 'corpus' directory
corpus_path = os.path.join(kaggle_reviews_path, 'corpus')
corpus_files = os.listdir(corpus_path)
corpus_files

In [ ]:
import pandas as pd

# Load the train.tsv file
train_data_path = os.path.join(corpus_path, 'train.tsv')
train_df = pd.read_csv(train_data_path, sep='\t')  # Assuming it's tab-separated based on the .tsv extension

train_df.head()


The train.tsv file has been successfully loaded into a DataFrame. Here's a look at its structure:

PhraseId: An identifier for each phrase.
SentenceId: An identifier for each sentence.
Phrase: The text of the phrase or sentence.
Sentiment: The sentiment classification for each phrase.

In [ ]:
import pandas as pd

# Load the train.tsv file
test_data_path = os.path.join(corpus_path, 'test.tsv')
test_df = pd.read_csv(test_data_path, sep='\t')  # Assuming it's tab-separated based on the .tsv extension

test_df.head()


The test.tsv file has been successfully loaded into a DataFrame. Here's a look at its structure:

PhraseId: An identifier for each phrase.

SentenceId: An identifier for each sentence.

Phrase: The text of the phrase or sentence.

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.Sentiment.value_counts()

In [ ]:
test_df.isnull().sum()

Here are the histograms displaying the distribution of word counts for each sentiment category in the training data:

Very Negative: Histogram shows the distribution of word counts for reviews labeled as very negative, with an average word count displayed.

Negative: Similar plot for negative sentiment.

Neutral: Shows the neutral reviews and their word counts.

Positive: Visualization for positive reviews.

Very Positive: Depicts the word count distribution for very positive reviews.

#Exploratory Data Analysis

In [ ]:
#Categorizing the ratings from 0 to 5 as Negative, Somewhat Negative, Neutral, Somewhat positive, Positive, Failed
Sentiment_phrase =[]

for row in train_df['Sentiment']:
    if row == 0:
        Sentiment_phrase.append('negative')
    elif row == 1:
        Sentiment_phrase.append('somewhat negative')
    elif row == 2:
        Sentiment_phrase.append('neutral')
    elif row == 3:
        Sentiment_phrase.append('somewhat positive')
    elif row == 4:
        Sentiment_phrase.append('positive')
    else:
        Sentiment_phrase.append('Failed')


train_df['Sentiment_phrase'] = Sentiment_phrase

In [ ]:
#Depicting the type of sentiments


# Calculate the word count for each phrase in the training data
train_df['WordCount'] = train_df['Phrase'].apply(lambda x: len(x.split()))

# Define sentiment categories for better labeling
sentiment_labels = {0: 'Very Negative', 1: 'Negative', 2: 'Neutral', 3: 'Positive', 4: 'Very Positive'}

# Create histograms for each sentiment category
fig, axes = plt.subplots(3, 2, figsize=(12, 18), sharex=True, sharey=True)
axes = axes.flatten()

for sentiment, ax in zip(sorted(sentiment_labels.keys()), axes):
    data = train_df[train_df['Sentiment'] == sentiment]['WordCount']
    ax.hist(data, bins=30, color='skyblue', alpha=0.7)
    ax.set_title(f'{sentiment_labels[sentiment]} (Sentiment {sentiment}) - Mean Words: {data.mean():.2f}')
    ax.set_xlabel('Word Count')
    ax.set_ylabel('Frequency')
    ax.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


5 histogram shows that the distribution behaves like a negative exponential function, decreasing significantly as the x-axis increases. The longest sentence in the expression column corresponds to the 'Negative Comments' class and seems to consist of around 52 words, now using the max() function the longest sentence can be found

In [ ]:
#maximum number of words found in a phrase
train_df['Phrase'].str.split().map(lambda x: len(x)).max()

In [ ]:
#Maximum number of words found in a particular sentence
train_df['Sentiment_phrase'].value_counts()

In [ ]:
#count of sentiment phrases
train_df['Sentiment_phrase'].value_counts().sort_index().plot(kind='bar', color= 'green')
plt.ylabel('Sentiment Phrase Count')
plt.xlabel('Sentiment')
plt.title('Count of Sentiments');

In [ ]:
print("--------------------------------------------------------------------")
dfff=pd.DataFrame(train_df['Phrase'].str.split().map(lambda x: len(x))>=20)
print('Number of sentences which contain more than 20 words: ', dfff.loc[dfff['Phrase']==True].shape[0])
print(' ')
print("--------------------------------------------------------------------")
dfff=pd.DataFrame(train_df['Phrase'].str.split().map(lambda x: len(x))>=30)
print('Number of sentences which contain more than 30 words: ', dfff.loc[dfff['Phrase']==True].shape[0])
print(' ')
print("--------------------------------------------------------------------")
dfff=pd.DataFrame(train_df['Phrase'].str.split().map(lambda x: len(x))>=40)
print('Number of sentences which contain more than 40 words: ', dfff.loc[dfff['Phrase']==True].shape[0])
print(' ')
print("--------------------------------------------------------------------")
dfff=pd.DataFrame(train_df['Phrase'].str.split().map(lambda x: len(x))>=50)
print('Number of sentences which contain more than 50 words: ', dfff.loc[dfff['Phrase']==True].shape[0])
print(' ')
print("--------------------------------------------------------------------")
dfff=pd.DataFrame(train_df['Phrase'].str.split().map(lambda x: len(x))==52)
print('Number of sentences which contain 52 words: ', dfff.loc[dfff['Phrase']==True].shape[0])
print(' ')
print("--------------------------------------------------------------------")

In [ ]:
train_df.columns

In [ ]:
# Calculate the number of phrases per sentence
phrases_per_sentence = train_df.groupby('SentenceId')['PhraseId'].count()

# Plotting the distribution of phrases per sentence
plt.figure(figsize=(8, 5))
phrases_per_sentence.hist(bins=30, color='lightcoral', edgecolor='black')
plt.title('Distribution of Phrases per Sentence')
plt.xlabel('Number of Phrases')
plt.ylabel('Frequency of Sentences')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
#Depiciting the number of bi-grams, and tri-grams
from sklearn.feature_extraction.text import CountVectorizer

def get_top_ngrams(corpus, n=None, ngram_range=(2,2)):
    vec = CountVectorizer(ngram_range=ngram_range, stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]

# Get the top 10 bi-grams and tri-grams
top_bi_grams = get_top_ngrams(train_df['Phrase'], n=10, ngram_range=(2,2))
top_tri_grams = get_top_ngrams(train_df['Phrase'], n=10, ngram_range=(3,3))

top_bi_grams, top_tri_grams


In [ ]:
# Extract phrases for negative, neutral, and positive sentiments
negative_phrases = train_df[train_df['Sentiment'] == 1]['Phrase'].sample(5, random_state=1).tolist()
neutral_phrases = train_df[train_df['Sentiment'] == 2]['Phrase'].sample(5, random_state=1).tolist()
positive_phrases = train_df[train_df['Sentiment'] == 3]['Phrase'].sample(5, random_state=1).tolist()

negative_phrases, neutral_phrases, positive_phrases


In [ ]:
#Representation of a wordcloud

from wordcloud import WordCloud

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(train_df['Phrase']))

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Phrases')
plt.show()


##Text Preprocessing

In [ ]:
X = train_df['Phrase'].values
y = train_df['Sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3 , stratify = y, random_state = 42)

##Split Train and Test Data

In [ ]:
from sklearn.model_selection import train_test_split
X, y = train_df.drop(columns=['Sentiment']), train_df['Sentiment']
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, stratify=y)

##Train Bag of Words Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def tokenizer(text):
    # Use regular expressions to match only word characters
    # This will split the text into words and remove punctuation
    return re.findall(r'\w+', text.lower())  # Convert to lowercase and find all word characters

cv = CountVectorizer(tokenizer=tokenizer)
# Presuming X_train['Phrase'] is a column of text data from your DataFrame
token_counts = cv.fit_transform(X_train['Phrase'])

In [ ]:
token_counts.shape

In [ ]:
X_train = pd.DataFrame.sparse.from_spmatrix(token_counts, columns=cv.get_feature_names_out())

#Modeling

##Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nbc = MultinomialNB()
#Fitting the classifier to the training data
nbc.fit(X_train, y_train)

In [ ]:
# Evaluating the performance of the trained Multinomial Naive Bayes classifier on the training data
nbc.score(X_train, y_train)

In [ ]:
#Transforms the phrases from the validatoin dataset into a feature matrix
val_counts = cv.transform(X_validate['Phrase'])

In [ ]:
#converting it into a pandas dataframe then uses the dataframe as inout to make predictions with the trained Naive Bayes classifier
X_validate = pd.DataFrame.sparse.from_spmatrix(val_counts, columns=cv.get_feature_names_out())
y_pred = nbc.predict(X_validate)

In [ ]:
#calculating the accuracy predictions, precision for predictions, recall for predictions and F1 score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def print_performance(clf, X, y):
    y_pred = clf.predict(X)

    accuracy = accuracy_score(y_pred, y)
    print("Accuracy: ", accuracy)
    print("Precision:", precision_score(y_pred, y, average='weighted'))
    print("Recall:   ", recall_score(y_pred, y, average='weighted'))
    print("F1 Score: ", f1_score(y_pred, y, average='weighted'))

    cm = confusion_matrix(y, y_pred) #creating the confusion matrix

    #confusion matrix heatmap
    plt.figure(figsize=(9,9))
    sns.heatmap(cm, annot=True, fmt="d", linewidths=.5, square = True, cmap = 'Blues_r');
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');

    #Displaying the acuuracy score on the heatmap
    all_sample_title = 'Accuracy Score: {0}'.format(accuracy)
    plt.title(all_sample_title, size = 15);

In [ ]:
print_performance(nbc, X_validate, y_validate)

##Decision Tree Classifier

In [ ]:
#Predicting the target values for the validation set and displaying the accuracy, precision, recall, F1 score and plot a confusion matrix
from sklearn.tree import DecisionTreeClassifier

dtf = DecisionTreeClassifier()
dtf.fit(X_train, y_train)

print_performance(dtf, X_validate, y_validate)

##TPU code

In [ ]:
!pip install transfomers

In [ ]:
import tensorflow as tf # importing the TensorFlow library
tpu = tf.distribute.cluster_resolver.TPUClusterResolver() #attempts to detect the TPU and establish a connection automatically.
tf.config.experimental_connect_to_cluster(tpu)# TensorFlow graph to the detected TPU cluster
tf.tpu.experimental.initialize_tpu_system(tpu)# Initializes the TPU system, configuring it for use by TensorFlow
strategy = tf.distribute.TPUStrategy(tpu) #Creating a TPUStrategy object

In [ ]:
#tensorflow libraries
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = train_df[['Phrase', 'Sentiment']]

# Set your model output as categorical and save in new label col
data['Sentiment_label'] = pd.Categorical(data['Sentiment'])

# Transform your output to numeric
data['Sentiment'] = data['Sentiment_label'].cat.codes

In [ ]:
data_train, data_val = train_test_split(data, test_size=0.1, stratify=data['Sentiment_label'], random_state=42)

print("Training Data Sentiment_label Distribution:")
print(data_train['Sentiment_label'].value_counts(normalize = True))

print("\nValidation Data Sentiment_label Distribution:")
print(data_val['Sentiment_label'].value_counts(normalize = True))

###BERT (Bidirectional Encoder Representations from Transformers)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
warnings.filterwarnings("ignore", category=FutureWarning, module="tensorflow")
warnings.filterwarnings("ignore", message="SetPriority unimplemented for this stream.", category=Warning)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [ ]:
# with TPU
from transformers import TFBertModel, BertTokenizer, BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import TFAutoModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Max length of tokens
max_length = 45

def create_model(max_length):
    # Load the MainLayer
    bert = TFAutoModel.from_pretrained('bert-base-uncased')

    # Build your model input
    input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
    inputs = {'input_ids': input_ids}

    # Load the Transformers BERT model as a layer in a Keras model
    bert_model = bert(inputs)[1]
    dropout = Dropout(0.1, name='pooled_output')  # Assuming config.hidden_dropout_prob is 0.1
    pooled_output = dropout(bert_model, training=False)

    # Then build your model output
    Sentiments = Dense(units=len(data_train.Sentiment_label.value_counts()),
                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
                       name='Sentiment')(pooled_output)
    outputs = {'Sentiment': Sentiments}
    # And combine it all in a model object
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='BERT_MultiClass')

    return model

# Create model inside the strategy scope
with strategy.scope():
    model = create_model(max_length)

    # Set an optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-05, epsilon=1e-08, clipnorm=1.0)

    # Set loss and metrics
    loss = {'Sentiment': tf.keras.losses.CategoricalCrossentropy(from_logits=True)}

    # Compile the model inside the strategy scope
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    # Print model summary
    model.summary()


In [ ]:
# Ready output data for the model
y_train = tf.keras.utils.to_categorical(data_train['Sentiment'])

# Tokenize the input (takes some time)
x_train = tokenizer(
    text=data_train['Phrase'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True)

y_val = tf.keras.utils.to_categorical(data_val['Sentiment'])
x_val = tokenizer(
    text=data_val['Phrase'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True)

# Fit the model
history = model.fit(
    x={'input_ids': x_train['input_ids']},
    y={'Sentiment': y_train},
    validation_data=({'input_ids': x_val['input_ids']}, {'Sentiment': y_val}),
    batch_size=32,
    epochs=2,
    verbose=1)

In [ ]:
#Evaluating the model using the validation data
model_eval = model.evaluate(
    x={'input_ids': x_val['input_ids']},
    y={'Sentiment': y_val}
)

In [ ]:
#Making predictions using the model on the validation datasetl
y_val_predicted = model.predict(
    x={'input_ids': x_val['input_ids']},
)
y_val_predicted['Sentiment']

In [ ]:
y_val_pred_max=[np.argmax(i) for i in y_val_predicted['Sentiment']]

In [ ]:
# The index corresponds to the class with the highest predicted probability
y_val_actual_max=[np.argmax(i) for i in y_val]

In [ ]:
#producing the classification report
from sklearn.metrics import classification_report

report = classification_report(y_val_pred_max, y_val_actual_max)

print(report)

validation set:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_val_pred_max, y_val_actual_max), display_labels=np.unique(y_val_actual_max))
disp.plot(cmap='Blues')
plt.grid(False)

In [ ]:
x_test = tokenizer(
          text=test_df['Phrase'].astype(str).to_list(),
          add_special_tokens=True,
          max_length=max_length,
          truncation=True,
          padding=True,
          return_tensors='tf',
          return_token_type_ids = False,
          return_attention_mask = False,
#is_split_into_words=True,
          verbose = True)

In [ ]:
label_predicted = model.predict(
    x={'input_ids': x_test['input_ids']},
)


In [ ]:
label_predicted['Sentiment']

In [ ]:
label_pred_max=[np.argmax(i) for i in label_predicted['Sentiment']]

In [ ]:
#Predicting the outputs from the validation set using model
from sklearn.metrics import f1_score, recall_score, average_precision_score

y_val_pred_max = model.predict(x_val['input_ids'])['Sentiment']  # 'Sentiment' çıkışını alın

y_pred = np.argmax(y_val_pred_max, axis=1)

y_pred_proba = tf.nn.softmax(y_val_pred_max)

# Calculating average precision score, which summarizes the precision-recall curve as the weighted mean of precisions at each threshold
BERT_AP = average_precision_score(y_val, y_pred_proba)
# Calculating the weighted F1 score, which considers both precision and recall to balance the contributions of false positives and false negatives
BERT_f1 = f1_score(np.argmax(y_val, axis=1), y_pred, average='weighted')
# Calculating the weighted recall, which is the ability of the classifier to find all the positive samples, weighted by the prevalence of each class
BERT_rec = recall_score(np.argmax(y_val, axis=1), y_pred, average='weighted')
# Printing out the calculated metrics
print("Average Precision:", BERT_AP)
print("Weighted F1 Score:", BERT_f1)
print("Weighted Recall:", BERT_rec)

In [ ]:
#creating a summary report for all the models accuracy rate.
from prettytable import PrettyTable

x = PrettyTable()

x.field_names = ['Model', 'Accuracy']
x.add_row(['Naive Bayes', '60.8%'])
x.add_row(['Decision Tree', '59.5%'])
x.add_row(['BERT', '68%'])
print(x)